In [1]:
import cv2
import numpy as np
import time
import HandTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
wcam, hcam = 640, 480

In [3]:
vid = cv2.VideoCapture(0)
vid.set(3, wcam)
vid.set(4, hcam)

pTime = 0

detector = htm.HandDetector(detection_confidence=0.7)


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
# volume.GetMute()
# volume.GetMasterVolumeLevel()
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]



while True:
    ret, image = vid.read()
    # detecting the hands    
    image = detector.findHands(image)
    
    # detecting the points of the landmarks
    lm_list = detector.findPosition(image, draw=False)
    if len(lm_list)!=0:
        # NOTE THAT WE ARE ONLY CONCERNED WITH LANDMARK NUMBER 4 AND 8 TO CONTROL THE VOLUME
        # print(lm_list[4], lm_list[8])   
        
        x1,y1 = lm_list[4][1], lm_list[4][2]
        x2,y2 = lm_list[8][1], lm_list[8][2]
        
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        
        # Drawing circle on the 4th and 8th landmark point
        cv2.circle(image, (x1,y1), 10, (0,255,0), cv2.FILLED)
        cv2.circle(image, (x2,y2), 10, (0,255,0), cv2.FILLED)
        
        # joining the two circles
        cv2.line(image, (x1,y1), (x2,y2), (0,255,0), 3)
        
        # drawing a circle on the mid of the line
        cv2.circle(image, (cx,cy), 14, (255,0,255), cv2.FILLED)
        
        # distance between the two landmarks
        length = math.hypot(x2-x1, y2-y1)
        # print(length)
        # length is in between 15 to 290
        
        vol = np.interp(length, [15,290], [min_vol, max_vol])
        # volume = volume.astype(int)
        # print(volume.dtype) - int32
        
        # -65 implies minimum volume and 0 implies max volume
        
        # setting the volume
        volume.SetMasterVolumeLevel(vol, None)
        
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    
    cv2.putText(image, f'FPS: {int(fps)}', (40,40), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3)
    
    cv2.imshow('Image', image)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break



vid.release()
cv2.destroyAllWindows()